In [12]:
import sys
import pandas as pd
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent))
from utils.utils import save_experiment, train_and_evaluate_linear_svm

In [9]:
train_data_path = r"c:\Users\konst\Documents\FoR_dataset\for-norm\for-norm\training_features.parquet"
test_data_path= r"c:\Users\konst\Documents\FoR_dataset\for-norm\for-norm\testing_features.parquet"
svc_params = {
            "C": 100,
            "class_weight": "balanced",
            "max_iter": 20000,
            "random_state": 42
        }
pipeline, metrics, svc_params, feature_names, metadata_extra = train_and_evaluate_linear_svm(train_data_path, test_data_path, svc_params)

In [10]:
print(metrics)

{'accuracy': 0.6963746223564955, 'precision': 0.7715736040609137, 'recall': 0.5772151898734177, 'f1': 0.6603910209992759}


In [11]:
save_experiment(
    model=pipeline,
    metrics=metrics,
    experiment_dir=r"C:\Users\konst\Documents\FoR_dataset\for-norm\for-norm\experiments",
    model_params=svc_params,
    feature_names=feature_names,
    metadata_extra=metadata_extra,
)


Experiment saved to: C:\Users\konst\Documents\FoR_dataset\for-norm\for-norm\experiments\exp_20251227_184835


'C:\\Users\\konst\\Documents\\FoR_dataset\\for-norm\\for-norm\\experiments\\exp_20251227_184835'

In [21]:
train_data_path = r"c:\Users\konst\Documents\FoR_dataset\for-norm\for-norm\training_features.parquet"
test_data_path= r"c:\Users\konst\Documents\FoR_dataset\for-norm\for-norm\testing_features.parquet"

train_df = pd.read_parquet(train_data_path)
train_df.dropna(inplace=True)
test_df = pd.read_parquet(test_data_path)
test_df.dropna(inplace=True)


rows_with_nas_train = train_df.isna().any(axis=1).sum()
rows_with_nas_test = test_df.isna().any(axis=1).sum()

print(f"Training rows with NaNs: {rows_with_nas_train}")
print(f"Testing rows with NaNs: {rows_with_nas_test}")

Training rows with NaNs: 0
Testing rows with NaNs: 0
